<a href="https://colab.research.google.com/github/hsshih/Kaggle_Galaxy_Classification/blob/main/Galaxy_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalMaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.models import model_from_json


In [ ]:
def get_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (2, 2), input_shape = input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size =(2, 2)))
    
    model.add(Conv2D(32, (2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size =(2, 2)))
    
    model.add(Conv2D(64, (2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size =(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return(model)


In [ ]:
def main():

    img_width, img_height = 212, 212

    train_data_dir = 'data/train'
    validation_data_dir = 'data/test'
    nb_train_samples = 400
    nb_validation_samples = 100
    epochs = 10
    batch_size = 16

    if K.image_data_format() == 'channels_first':
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)

    model = get_model(input_shape)

    model.compile(loss ='binary_crossentropy',
                  optimizer ='rmsprop',
                  metrics =['accuracy'])

    train_datagen = ImageDataGenerator(
                                       rescale = 1. / 255,
                                       shear_range = 0.2,
                                       zoom_range = 0.2,
                                       horizontal_flip = True)

    test_datagen = ImageDataGenerator(rescale = 1. / 255)

    train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                        target_size =(img_width, img_height),
                                                        batch_size = batch_size, class_mode ='binary')

    validation_generator = test_datagen.flow_from_directory(
                                                            validation_data_dir,
                                                            target_size =(img_width, img_height),
                                                            batch_size = batch_size, class_mode ='binary') 

    model.fit_generator(train_generator, 
                        steps_per_epoch = nb_train_samples // batch_size, 
                        epochs = epochs, validation_data = validation_generator, 
                        validation_steps = nb_validation_samples // batch_size) 


    # serialize model to JSON
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")

    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")

    # evaluate loaded model on test data
    loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    score = loaded_model.evaluate(validation_generator, verbose=0)
    print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))




if __name__ == "__main__":
    main()
